In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from matplotlib.patches import Patch
import matplotlib.gridspec as gridspec
from PIL import Image
from scipy import ndimage

# --- KONFIGURASI ---
# Ganti path ini dengan lokasi file Excel/CSV kamu yang sebenarnya
file_path = r"E:\rsa-lenovo\komparasi\hasil analisa citra.xlsx"
image_folder = r"E:\rsa-lenovo\komparasi"


def load_and_preprocess_data(path):
    """
    Membaca data dan mengekstrak informasi Brand dan Body Part dari Filename.
    """
    if path.endswith(".csv"):
        df = pd.read_csv(path)
    else:
        df = pd.read_excel(path)

    def extract_info(filename):
        try:
            parts = filename.split("_")
            body_part = parts[0]
            brand = parts[1]
            return body_part, brand
        except:
            return "Unknown", "Unknown"

    df[["Body Part", "Brand"]] = df["Filename"].apply(
        lambda x: pd.Series(extract_info(x))
    )
    df["Brand"] = df["Brand"].str.lower().str.strip()

    return df


def load_image_safe(image_path):
    """Load image dengan berbagai ekstensi yang mungkin."""
    extensions = ['', '.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp']
    
    for ext in extensions:
        try_path = image_path if ext == '' else os.path.splitext(image_path)[0] + ext
        if os.path.exists(try_path):
            try:
                img = Image.open(try_path)
                if img.mode != 'L':
                    img = img.convert('L')
                return np.array(img)
            except Exception as e:
                continue
    return None


def extract_profile(image_array):
    """Ekstrak profile intensitas dari citra (horizontal mid-line)."""
    if image_array is None:
        return None
    mid_row = image_array.shape[0] // 2
    return image_array[mid_row, :]


def validate_data(df):
    """Validasi data untuk memastikan kelengkapan sebelum plotting."""
    print("\n" + "=" * 80)
    print("🔍 VALIDASI DATA SEBELUM PLOTTING")
    print("=" * 80 + "\n")
    
    metrics = ["SNR", "CNR", "MTF50", "FWHM"]
    body_parts = df["Body Part"].unique()
    brands = df["Brand"].unique()
    
    validation_report = {
        "complete": [],
        "incomplete": [],
        "missing_brand": [],
        "missing_metrics": {}
    }
    
    print(f"📊 Total Body Parts: {len(body_parts)}")
    print(f"🏷️  Brands Found: {', '.join([b.capitalize() for b in brands])}")
    print(f"📈 Metrics: {', '.join(metrics)}\n")
    print("-" * 80)
    
    for part in body_parts:
        subset = df[df["Body Part"] == part]
        brands_in_part = subset["Brand"].unique()
        
        print(f"\n📦 {part}:")
        print(f"   Brands: {', '.join([b.capitalize() for b in brands_in_part])} ({len(brands_in_part)} brand(s))")
        
        if len(brands_in_part) < 2:
            validation_report["incomplete"].append(part)
            validation_report["missing_brand"].append({
                "body_part": part,
                "brands": list(brands_in_part),
                "missing": [b for b in brands if b not in brands_in_part]
            })
            print(f"   ⚠️  WARNING: Hanya ada {len(brands_in_part)} brand!")
            continue
        
        missing_info = []
        for metric in metrics:
            missing_count = subset[metric].isna().sum()
            if missing_count > 0:
                missing_info.append(f"{metric}: {missing_count} missing")
                if part not in validation_report["missing_metrics"]:
                    validation_report["missing_metrics"][part] = []
                validation_report["missing_metrics"][part].append({
                    "metric": metric, "count": missing_count
                })
        
        if missing_info:
            print(f"   ⚠️  Missing values: {', '.join(missing_info)}")
        else:
            print(f"   ✅ Data lengkap!")
            validation_report["complete"].append(part)
        
        for brand in brands_in_part:
            count = len(subset[subset["Brand"] == brand])
            print(f"      • {brand.capitalize()}: {count} samples")
    
    print("\n" + "=" * 80)
    print("📋 RINGKASAN VALIDASI:")
    print("=" * 80)
    print(f"✅ Lengkap: {len(validation_report['complete'])} - {', '.join(validation_report['complete']) if validation_report['complete'] else '-'}")
    print(f"⚠️  Tidak lengkap: {len(validation_report['incomplete'])} - {', '.join(validation_report['incomplete']) if validation_report['incomplete'] else '-'}")
    print("=" * 80 + "\n")
    
    return validation_report


def generate_comparison_plots(df, output_folder, validation_report, img_folder):
    """Membuat plot perbandingan bar chart untuk semua metrik dengan profile intensitas."""
    metrics = ["SNR", "CNR", "MTF50", "FWHM"]
    sns.set_theme(style="whitegrid")
    
    palette = {
        "canon": "#2E86AB",
        "madeena": "#A23B72",
    }

    print(f"📂 Output folder: {output_folder}\n")
    print("🎨 Memulai plotting...\n")

    body_parts_to_plot = validation_report['complete']
    
    if not body_parts_to_plot:
        print("❌ Tidak ada body part dengan data lengkap!")
        return

    for idx, part in enumerate(body_parts_to_plot, 1):
        print(f"[{idx}/{len(body_parts_to_plot)}] {part}...", end=" ")
        
        subset = df[df["Body Part"] == part]
        brands_available = subset["Brand"].unique()

        # === IMPROVED LAYOUT ===
        fig = plt.figure(figsize=(18, 12))
        gs = gridspec.GridSpec(3, 4, figure=fig, hspace=0.35, wspace=0.35, 
                               height_ratios=[1, 1, 1.3], width_ratios=[1, 1, 1, 0.8])
        fig.suptitle(f"Analisis Perbandingan Citra: {part}", 
                     fontsize=20, fontweight="bold", y=0.97)

        # === BAR CHARTS (2x2 layout) ===
        bar_positions = [(0, 0), (0, 1), (1, 0), (1, 1)]
        for i, metric in enumerate(metrics):
            row, col = bar_positions[i]
            ax = fig.add_subplot(gs[row, col])
            
            stats_data = []
            for brand in brands_available:
                brand_data = subset[subset["Brand"] == brand][metric]
                stats_data.append({
                    "Brand": brand.capitalize(),
                    "Mean": brand_data.mean(),
                    "Std": brand_data.std() if len(brand_data) > 1 else 0,
                    "Count": len(brand_data)
                })
            
            stats_df = pd.DataFrame(stats_data)
            
            if metric == "FWHM":
                min_val, max_val = stats_df["Mean"].min(), stats_df["Mean"].max()
                colors = ["#28a745" if m == min_val else "#dc3545" if m == max_val else "#ffc107" 
                          for m in stats_df["Mean"]]
            else:
                colors = [palette.get(b.lower(), "#888888") for b in stats_df["Brand"]]
            
            bars = ax.bar(stats_df["Brand"], stats_df["Mean"], yerr=stats_df["Std"],
                          color=colors, capsize=6, alpha=0.85, edgecolor="black", linewidth=1.2)
            
            title_suffix = " (↓ Better)" if metric == "FWHM" else ""
            ax.set_title(metric + title_suffix, fontsize=12, fontweight="bold", pad=6)
            ax.set_xlabel("")
            ax.set_ylabel("Value", fontsize=9)
            ax.grid(axis='y', alpha=0.3, linestyle='--')
            ax.tick_params(axis='both', labelsize=9)
            
            for bar, mean, std in zip(bars, stats_df["Mean"], stats_df["Std"]):
                height = bar.get_height()
                label_text = f'{mean:.2f}' + (f'\n±{std:.2f}' if std > 0 else '')
                ax.text(bar.get_x() + bar.get_width()/2., height + std + 0.02*height,
                        label_text, ha='center', va='bottom', fontsize=8, fontweight='bold')

        # === LEGEND (Right side, top) ===
        ax_legend = fig.add_subplot(gs[0, 2:4])
        ax_legend.axis('off')
        
        legend_elements = [Patch(facecolor=palette.get(b.lower(), "#888888"), 
                                 edgecolor='black', label=b.capitalize()) 
                          for b in brands_available]
        ax_legend.legend(handles=legend_elements, loc='center', fontsize=11,
                        title="Alat", title_fontsize=12, frameon=True, 
                        fancybox=True, shadow=True, ncol=2)

        # === METRIC EXPLANATION (Right side, middle) ===
        ax_info = fig.add_subplot(gs[1, 2:4])
        ax_info.axis('off')
        
        metric_text = (
            "METRIK:\n"
            "• SNR: Signal-to-Noise (↑ baik)\n"
            "• CNR: Contrast-to-Noise (↑ baik)\n"
            "• MTF50: Ketajaman (↑ baik)\n"
            "• FWHM: Lebar puncak (↓ baik)"
        )
        ax_info.text(0.5, 0.5, metric_text, transform=ax_info.transAxes,
                    ha='center', va='center', fontsize=9, family='monospace',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='lightyellow', alpha=0.6))

        # === PROFILE PLOT (Bottom, spanning 3 columns) ===
        ax_profile = fig.add_subplot(gs[2, 0:3])
        
        profiles = {}
        for brand in brands_available:
            brand_subset = subset[subset["Brand"] == brand]
            if len(brand_subset) > 0:
                filename = brand_subset.iloc[0]["Filename"]
                image_path = os.path.join(img_folder, filename)
                img_array = load_image_safe(image_path)
                if img_array is not None:
                    profile = extract_profile(img_array)
                    if profile is not None:
                        profiles[brand] = profile
        
        if profiles:
            for brand, profile in profiles.items():
                color = palette.get(brand.lower(), "#888888")
                ax_profile.plot(profile, label=brand.capitalize(), color=color, 
                               linewidth=1.8, alpha=0.85)
            
            ax_profile.set_title("Profile Intensitas Gray Value (Horizontal Mid-Line)", 
                                fontsize=12, fontweight="bold", pad=8)
            ax_profile.set_xlabel("Distance (pixels)", fontsize=10)
            ax_profile.set_ylabel("Gray Value", fontsize=10)
            ax_profile.grid(True, alpha=0.3, linestyle='--')
            ax_profile.legend(loc='upper right', fontsize=9)
            ax_profile.tick_params(axis='both', labelsize=9)
            
            if 'canon' in profiles and 'madeena' in profiles:
                c_range = profiles['canon'].max() - profiles['canon'].min()
                m_range = profiles['madeena'].max() - profiles['madeena'].min()
                c_std, m_std = np.std(profiles['canon']), np.std(profiles['madeena'])
                
                info_text = f"Canon: Range={c_range:.0f}, Std={c_std:.1f}  |  Madeena: Range={m_range:.0f}, Std={m_std:.1f}"
                ax_profile.text(0.02, 0.97, info_text, transform=ax_profile.transAxes,
                               fontsize=8, va='top',
                               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.6))
        else:
            ax_profile.text(0.5, 0.5, "⚠️ Profile tidak tersedia\n(Gambar tidak ditemukan)",
                           ha='center', va='center', fontsize=11, transform=ax_profile.transAxes)
            ax_profile.set_title("Profile Intensitas", fontsize=12, fontweight="bold")

        # === CONCLUSION (Bottom right) ===
        ax_note = fig.add_subplot(gs[2, 3])
        ax_note.axis('off')
        
        conclusion_text = (
            "KESIMPULAN:\n\n"
            "Perbedaan teknologi detektor\n"
            "membuat kedua alat tidak bisa\n"
            "dibandingkan langsung.\n\n"
            "Namun berdasarkan parameter\n"
            "OBJEKTIF, Madeena menunjukkan\n"
            "performa lebih baik dalam:\n"
            "  • SNR (noise lebih rendah)\n"
            "  • CNR (kontras lebih baik)\n"
            "Profile plot menunjukkan:\n"
            "  • Madeena: garis lebih smooth\n"
            "    (noise rendah, konsisten)\n"
            "  • Canon: garis lebih bergerigi\n"
            "    (noise tinggi, tapi detail\n"
            "    tampak 'tajam' karena\n"
            "    pipeline non-linear)\n\n"
        )
        ax_note.text(0.5, 0.95, conclusion_text, transform=ax_note.transAxes,
                    ha='center', va='top', fontsize=8, family='monospace',
                    bbox=dict(boxstyle='round,pad=0.4', facecolor='lightcyan', alpha=0.6))

        # Simpan
        safe_filename = "".join([c for c in part if c.isalnum() or c == " "]).rstrip()
        file_name = f"{safe_filename}_comparison.png"
        save_path = os.path.join(output_folder, file_name)
        
        fig.savefig(save_path, bbox_inches="tight", dpi=300)
        plt.close(fig)
        
        print(f"✅ {file_name}")

    print("\n" + "=" * 80)
    print(f"🎉 Selesai! {len(body_parts_to_plot)} gambar tersimpan.")
    print("=" * 80 + "\n")


# --- EKSEKUSI PROGRAM ---
try:
    print("\n" + "=" * 80)
    print("🚀 PROGRAM ANALISIS PERBANDINGAN CITRA")
    print("=" * 80)
    
    print("\n📂 Membaca data...")
    df_result = load_and_preprocess_data(file_path)
    print(f"✅ Data: {len(df_result)} baris")

    print("\n📋 Preview Data:")
    print("-" * 80)
    display(df_result[["Filename", "Body Part", "Brand", "Score"]].head(10))
    
    validation_report = validate_data(df_result)
    
    if validation_report['complete']:
        base_dir = os.path.dirname(os.path.abspath(file_path))
        output_dir = os.path.join(base_dir, "hasil_plot_perbandingan")
        os.makedirs(output_dir, exist_ok=True)
        
        generate_comparison_plots(df_result, output_dir, validation_report, image_folder)
        
        print(f"\n📁 Output: {output_dir}")
    else:
        print("\n❌ Tidak dapat plotting - data tidak lengkap.\n")

except FileNotFoundError:
    print(f"\n❌ File tidak ditemukan: {file_path}\n")
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()


🚀 PROGRAM ANALISIS PERBANDINGAN CITRA

📂 Membaca data...
✅ Data: 48 baris

📋 Preview Data:
--------------------------------------------------------------------------------


,Filename,Body Part,Brand,Score
0,Ankle AP_canon_52kV_10mA_5s.jpg,Ankle AP,canon,0.363324
1,Ankle AP_madeena_80kV_50mA_0.4s.jpg,Ankle AP,madeena,0.474990
2,Ankle Lateral_canon_52kV_10mA_5s.jpg,Ankle Lateral,canon,0.432315
3,Ankle Lateral_madeena_80kV_50mA_0.4s.jpg,Ankle Lateral,madeena,0.418329
4,Antebrachi AP_canon_48kV_11.11mA_4.5s.jpg,Antebrachi AP,canon,0.210001
5,Antebrachi AP_madeena_70kV_50mA_0.2s.jpg,Antebrachi AP,madeena,0.592742
6,Antebrachi Lateral_canon_48kV_11.11mA_4.5s.jpg,Antebrachi Lateral,canon,0.385774
7,Antebrachi Lateral_madeena_70kV_50mA_0.2s.jpg,Antebrachi Lateral,madeena,0.462358
8,Cruris AP_canon_55kV_7.14mA_7s.jpg,Cruris AP,canon,0.399787
9,Cruris AP_madeena_80kV_50mA_0.5s.jpg,Cruris AP,madeena,0.495457



🔍 VALIDASI DATA SEBELUM PLOTTING

📊 Total Body Parts: 24
🏷️  Brands Found: Canon, Madeena
📈 Metrics: SNR, CNR, MTF50, FWHM

--------------------------------------------------------------------------------

📦 Ankle AP:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Ankle Lateral:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Antebrachi AP:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Antebrachi Lateral:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Cruris AP:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Cruris Lateral:
   Brands: Canon, Madeena (2 brand(s))
   ✅ Data lengkap!
      • Canon: 1 samples
      • Madeena: 1 samples

📦 Elbow Joint AP:
   Brands: